#  情感分析 - 詞頻矩陣
使用jieba對文本進行切詞，生成詞頻矩陣，並使用Naive Bayes模型進行情感分析。

## 準備資料

將切詞後的文本資料轉換為適合機器學習模型的格式，例如詞頻矩陣。


In [8]:
import pandas
# 載入資料集
df = pandas.read_csv('https://raw.githubusercontent.com/ywchiu/hp_ai_advanced_training/refs/heads/main/data/hp_reviews.csv')
# 顯示資料集前幾列
df.head()

,內容,情緒
0,HP就是穩！大廠牌品管有保證啦～不像某些雜牌三不五時就出包，我公司採購都指定HP或Dell，...,正面
1,原廠服務真的讚，上次要加記憶體，送去服務中心免費裝，反觀國產牌還要收費= = 而且HP的工程...,正面
2,買了Victus 15用來打game，外型低調不會被看出是電競筆電，爽！在咖啡廳用也不會太招...,正面
3,OmniBook超薄超輕，帶出門完全沒負擔，電池還能撐一整天👍 我每天通勤背著也不會肩膀痠，...,正面
4,黑五搶到HP 14吋只要236美金！！學生黨表示太香了～～ 規格雖然不是頂級但日常使用綽綽有...,正面


In [9]:
import jieba

# 對 '內容' 欄位進行 jieba 分詞並將結果儲存到新的 '切詞' 欄位中
df['切詞'] = df['內容'].apply(lambda x: list(jieba.cut(x)))

# 顯示包含新 '切詞' 欄位的 DataFrame
display(df.head())

,內容,情緒,切詞
0,HP就是穩！大廠牌品管有保證啦～不像某些雜牌三不五時就出包，我公司採購都指定HP或Dell，...,正面,"[HP, 就是, 穩, ！, 大廠, 牌品, 管有, 保證, 啦, ～, 不像, 某些, 雜..."
1,原廠服務真的讚，上次要加記憶體，送去服務中心免費裝，反觀國產牌還要收費= = 而且HP的工程...,正面,"[原廠, 服務, 真的, 讚, ，, 上次, 要加, 記憶體, ，, 送去, 服務, 中心,..."
2,買了Victus 15用來打game，外型低調不會被看出是電競筆電，爽！在咖啡廳用也不會太招...,正面,"[買, 了, Victus, , 15, 用來, 打, game, ，, 外型, 低調, ..."
3,OmniBook超薄超輕，帶出門完全沒負擔，電池還能撐一整天👍 我每天通勤背著也不會肩膀痠，...,正面,"[OmniBook, 超薄, 超輕, ，, 帶, 出門, 完全, 沒, 負擔, ，, 電池,..."
4,黑五搶到HP 14吋只要236美金！！學生黨表示太香了～～ 規格雖然不是頂級但日常使用綽綽有...,正面,"[黑五, 搶, 到, HP, , 14, 吋, 只要, 236, 美金, ！, ！, 學生..."


In [10]:
# 計算每個情緒類別的數量
sentiment_counts = df['情緒'].value_counts()

# 顯示計數結果
display(sentiment_counts)

,count
情緒,
正面,50
中立,50
負面,50


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# 將 '切詞' 欄位中的詞語列表轉換回字串
df['切詞字串'] = df['切詞'].apply(lambda x: ' '.join(x))

# 初始化並擬合 CountVectorizer
vectorizer = CountVectorizer()
term_frequency_matrix = vectorizer.fit_transform(df['切詞字串'])

# 顯示詞頻矩陣的形狀以確認已建立
print("詞頻矩陣的形狀:", term_frequency_matrix.shape)

詞頻矩陣的形狀: (150, 1602)


## 分割資料集

將資料分割成訓練集和測試集。


In [12]:
from sklearn.model_selection import train_test_split

# 定義特徵 (X) 和目標變數 (y)
X = term_frequency_matrix
y = df['情緒']

# 將資料分割成訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 印出分割後集合的形狀以驗證分割結果
print("X_train 的形狀:", X_train.shape)
print("X_test 的形狀:", X_test.shape)
print("y_train 的形狀:", y_train.shape)
print("y_test 的形狀:", y_test.shape)

X_train 的形狀: (112, 1602)
X_test 的形狀: (38, 1602)
y_train 的形狀: (112,)
y_test 的形狀: (38,)


## 訓練模型

使用訓練集訓練 Naive Bayes 模型。


In [13]:
from sklearn.naive_bayes import MultinomialNB

# 初始化一個 Multinomial Naive Bayes 模型
model = MultinomialNB()

# 使用訓練資料訓練模型
model.fit(X_train, y_train)

MultinomialNB()

## 評估模型

使用測試集評估模型的效能。


In [14]:
from sklearn.metrics import accuracy_score, classification_report

# 在測試集上進行預測
y_pred = model.predict(X_test)

# 計算並印出準確度
accuracy = accuracy_score(y_test, y_pred)
print(f"測試集上的準確度: {accuracy:.2f}")

# 生成並印出分類報告
report = classification_report(y_test, y_pred)
print("\n分類報告:")
print(report)

測試集上的準確度: 0.79

分類報告:
              precision    recall  f1-score   support

          中立       0.61      1.00      0.76        11
          正面       1.00      0.73      0.85        15
          負面       0.89      0.67      0.76        12

    accuracy                           0.79        38
   macro avg       0.83      0.80      0.79        38
weighted avg       0.85      0.79      0.79        38



In [16]:
from collections import Counter

# 根據情緒類別分組 DataFrame
grouped_sentiment = df.groupby('情緒')['切詞'].sum()

# 定義一個函式來找出每個情緒類別中最常見的詞語
def get_top_words(word_list, num_words=10):
    # 過濾掉空白字串和單字元的詞語 (可選)
    cleaned_words = [word for word in word_list if word.strip() and len(word.strip()) > 1]
    # 使用 Counter 計算詞頻並返回最常見的前 num_words 個詞語
    return Counter(cleaned_words).most_common(num_words)

# 獲取正面情緒的代表詞彙
top_positive_words = get_top_words(grouped_sentiment['正面'])
# 獲取中立情緒的代表詞彙
top_neutral_words = get_top_words(grouped_sentiment['中立'])
# 獲取負面情緒的代表詞彙
top_negative_words = get_top_words(grouped_sentiment['負面'])

print("正面情緒的代表詞彙:")
for word, count in top_positive_words:
    print(f"{word}: {count}")

print("\n中立情緒的代表詞彙:")
for word, count in top_neutral_words:
    print(f"{word}: {count}")

print("\n負面情緒的代表詞彙:")
for word, count in top_negative_words:
    print(f"{word}: {count}")

正面情緒的代表詞彙:
HP: 33
真的: 32
而且: 18
不會: 16
OMEN: 12
筆電: 10
AI: 9
效能: 8
方便: 8
不用: 7

中立情緒的代表詞彙:
HP: 60
不過: 19
筆電: 18
品牌: 11
很多: 11
市場: 10
有些: 10
可以: 10
還是: 9
這是: 8

負面情緒的代表詞彙:
HP: 30
問題: 15
一堆: 13
筆電: 10
品質: 10
真的: 10
螢幕: 9
根本: 9
不能: 9
設計: 8


以下是這段英文的正體中文翻譯：

---

## 摘要

### 資料分析主要發現

* 文本資料已成功轉換為詞頻矩陣，矩陣維度為 (150, 1602)，代表150筆文件與1602個獨特詞彙。
* 資料被拆分為訓練集與測試集，維度分別為：訓練特徵 (112, 1602)、測試特徵 (38, 1602)、訓練標籤 (112,) 與測試標籤 (38,)。
* 成功以訓練資料訓練出一個多項式朴素貝葉斯（Multinomial Naive Bayes）模型。
* 該訓練模型在測試集上達到0.79的準確率。
* 分類報告顯示模型在不同情緒分類上的表現有所差異：

  * 中立情緒具有極高的召回率（1.00），但精確率較低（0.61）。
  * 正向情緒則有完美的精確率（1.00），但召回率為0.73。
  * 負向情緒的精確率為0.89，召回率為0.67。

### 洞見與後續步驟

* 雖然整體準確率尚可，但模型在各情緒類別的表現差異明顯。建議進一步分析模型為何在中立情緒的精確率，以及正向與負向情緒的召回率上表現不佳。
* 可考慮探索其他文本表示方式（例如 TF-IDF）或不同的機器學習模型，以期提升分類效能，特別是針對樣本不平衡的情緒類別。


# 情感分析 - Word2Vec

In [21]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

## Text2Vec 特徵工程

使用 Text2Vec 將文本資料轉換為向量表示。

In [16]:
# 載入 Text2Vec 相關函式
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 這裡我們將使用 TF-IDF 作為 Text2Vec 的一種實現方式
# 也可以使用 Word2Vec 或其他更進階的 Text2Vec 方法
tfidf_vectorizer = TfidfVectorizer(max_features=1000) # 可以調整 max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(df['切詞字串'])

print("TF-IDF 矩陣的形狀:", tfidf_matrix.shape)

TF-IDF 矩陣的形狀: (150, 1000)


## 分割資料集 (Text2Vec)

將 Text2Vec 轉換後的資料分割成訓練集和測試集。

In [17]:
from sklearn.model_selection import train_test_split

# 定義特徵 (X) 和目標變數 (y)
X_text2vec = tfidf_matrix
y_text2vec = df['情緒']

# 將資料分割成訓練集和測試集
X_train_text2vec, X_test_text2vec, y_train_text2vec, y_test_text2vec = train_test_split(X_text2vec, y_text2vec, test_size=0.25, random_state=42)

# 印出分割後集合的形狀以驗證分割結果
print("X_train_text2vec 的形狀:", X_train_text2vec.shape)
print("X_test_text2vec 的形狀:", X_test_text2vec.shape)
print("y_train_text2vec 的形狀:", y_train_text2vec.shape)
print("y_test_text2vec 的形狀:", y_test_text2vec.shape)

X_train_text2vec 的形狀: (112, 1000)
X_test_text2vec 的形狀: (38, 1000)
y_train_text2vec 的形狀: (112,)
y_test_text2vec 的形狀: (38,)


## 訓練模型 (Text2Vec)

使用 Text2Vec 轉換後的訓練集訓練 Naive Bayes 模型。

In [18]:
from sklearn.naive_bayes import MultinomialNB

# 初始化一個 Multinomial Naive Bayes 模型
model_text2vec = MultinomialNB()

# 使用訓練資料訓練模型
model_text2vec.fit(X_train_text2vec, y_train_text2vec)

MultinomialNB()

## 評估模型 (Text2Vec)

使用 Text2Vec 轉換後的測試集評估模型的效能。

In [19]:
from sklearn.metrics import accuracy_score, classification_report

# 在測試集上進行預測
y_pred_text2vec = model_text2vec.predict(X_test_text2vec)

# 計算並印出準確度
accuracy_text2vec = accuracy_score(y_test_text2vec, y_pred_text2vec)
print(f"使用 Text2Vec 的測試集上的準確度: {accuracy_text2vec:.2f}")

# 生成並印出分類報告
report_text2vec = classification_report(y_test_text2vec, y_pred_text2vec)
print("\n使用 Text2Vec 的分類報告:")
print(report_text2vec)

使用 Text2Vec 的測試集上的準確度: 0.79

使用 Text2Vec 的分類報告:
              precision    recall  f1-score   support

          中立       0.58      1.00      0.73        11
          正面       1.00      0.73      0.85        15
          負面       1.00      0.67      0.80        12

    accuracy                           0.79        38
   macro avg       0.86      0.80      0.79        38
weighted avg       0.88      0.79      0.80        38



## 訓練模型 (SVM)

使用 Text2Vec 轉換後的訓練集訓練 Support Vector Machine (SVM) 模型。

In [20]:
from sklearn.svm import SVC

# 初始化一個 Support Vector Machine (SVM) 模型
# 可以嘗試不同的 kernel (如 'linear', 'rbf') 和 C 值
svm_model = SVC(kernel='linear', random_state=42)

# 使用訓練資料訓練模型
svm_model.fit(X_train_text2vec, y_train_text2vec)

SVC(kernel='linear', random_state=42)

## 評估模型 (SVM)

使用 Text2Vec 轉換後的測試集評估 SVM 模型的效能。

In [21]:
from sklearn.metrics import accuracy_score, classification_report

# 在測試集上進行預測
y_pred_svm = svm_model.predict(X_test_text2vec)

# 計算並印出準確度
accuracy_svm = accuracy_score(y_test_text2vec, y_pred_svm)
print(f"使用 Text2Vec 和 SVM 的測試集上的準確度: {accuracy_svm:.2f}")

# 生成並印出分類報告
report_svm = classification_report(y_test_text2vec, y_pred_svm)
print("\n使用 Text2Vec 和 SVM 的分類報告:")
print(report_svm)

使用 Text2Vec 和 SVM 的測試集上的準確度: 0.82

使用 Text2Vec 和 SVM 的分類報告:
              precision    recall  f1-score   support

          中立       0.65      1.00      0.79        11
          正面       1.00      0.73      0.85        15
          負面       0.90      0.75      0.82        12

    accuracy                           0.82        38
   macro avg       0.85      0.83      0.82        38
weighted avg       0.87      0.82      0.82        38



## 情感分析 - Word2Vec 分析摘要

我們使用了 **TF-IDF** 作為 Text2Vec 的一種實現方式，並結合了 **Naive Bayes** 和 **Support Vector Machine (SVM)** 模型進行情感分析。

**目前的分析結果 (基於 TF-IDF 特徵):**

*   **資料前處理:** 文本資料經過 Jieba 分詞處理，並轉換為 TF-IDF 矩陣，用於表示文本特徵。
*   **模型訓練與評估:** 我們分別訓練了 Naive Bayes 和 SVM 模型。
    *   Naive Bayes 模型在測試集上的準確率為 0.79。
    *   SVM 模型在測試集上的準確率為 0.82，表現略優於 Naive Bayes。
*   **模型表現:** 分類報告顯示，模型在不同情緒類別上的表現有所差異，尤其在中立情緒的精確率上仍有改進空間。

**關於 Word2Vec 分析:**

Word2Vec 是一種不同於 TF-IDF 的文本向量化方法。它不是簡單地計算詞頻或詞的重要性，而是透過訓練一個神經網路來學習詞語的語義資訊，將每個詞語映射到一個低維度的向量空間中。在這個向量空間中，語義相似的詞語會靠得比較近。

如果我們要使用 Word2Vec 進行情感分析，步驟會與使用 TF-IDF 有些不同：

1.  **訓練 Word2Vec 模型:** 使用我們的文本資料 (切詞後的文本) 來訓練一個 Word2Vec 模型。這個模型會學習每個詞語的向量表示。
2.  **生成文件向量:** 由於 Word2Vec 為每個詞語生成向量，我們需要一個方法將一篇文檔中的所有詞語向量組合成一個文檔向量。常見的方法包括對文檔中所有詞語的 Word2Vec 向量取平均或加權平均。
3.  **分割資料集:** 將生成的文檔向量和對應的情緒標籤分割成訓練集和測試集。
4.  **訓練分類模型:** 使用文檔向量作為特徵，情緒標籤作為目標變數，訓練一個分類模型 (例如 Naive Bayes, SVM, 或其他深度學習模型)。
5.  **評估模型:** 在測試集上評估模型的效能。

使用 Word2Vec 的優勢在於它可以捕捉詞語之間的語義關係，這對於理解文本的真正含義可能更有幫助，進而可能提升情感分析的效能，特別是對於那些在訓練數據中不常出現的詞語。


# 情感分析 - BERT
使用 Bert 模型對文本進行分類。

## 安裝 transformers 函式庫

安裝 `transformers` 函式庫，以便使用 Bert 模型。


In [22]:
!pip install transformers

## 載入 bert 模型和 tokenizer

載入預訓練的 Bert 模型和對應的 Tokenizer。


In [23]:
from transformers import BertModel, BertTokenizer

# 指定使用的預訓練模型名稱
model_name = 'bert-base-chinese'

# 實例化 Bert Tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# 實例化 Bert 模型
bert_model = BertModel.from_pretrained(model_name)

print(f"成功載入預訓練模型: {model_name}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

成功載入預訓練模型: bert-base-chinese


## 文本 tokenization

使用 Bert Tokenizer 對切詞後的文本資料進行 tokenization，將文本轉換為 Bert 模型所需的輸入格式。


In [24]:
# 創建一個函數來對文本列表進行編碼
def tokenize_text_list(text_list, tokenizer, max_length=128):
    tokenized_inputs = tokenizer(
        text_list,
        padding=True,          # 對較短的序列進行填充以達到 max_length
        truncation=True,       # 對較長的序列進行截斷以達到 max_length
        max_length=max_length, # 設定最大序列長度
        return_tensors='pt'    # 返回 PyTorch 張量
    )
    return tokenized_inputs

# 將 tokenization 函數應用到 '切詞字串' 列
tokenized_texts = tokenize_text_list(df['切詞字串'].tolist(), tokenizer, max_length=128)

# 印出 tokenized_texts 的鍵和形狀以驗證結果
print("Tokenized texts keys:", tokenized_texts.keys())
print("input_ids shape:", tokenized_texts['input_ids'].shape)
print("attention_mask shape:", tokenized_texts['attention_mask'].shape)
print("token_type_ids shape:", tokenized_texts['token_type_ids'].shape)

Tokenized texts keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids shape: torch.Size([150, 78])
attention_mask shape: torch.Size([150, 78])
token_type_ids shape: torch.Size([150, 78])


## 準備資料集

將 tokenized 的資料分割成訓練集和測試集，並轉換為 PyTorch 或 TensorFlow 的張量格式，以便輸入到 Bert 模型。


In [27]:
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import LabelEncoder

# Define features (X) and target variable (y)
X = [tokenized_texts['input_ids'], tokenized_texts['attention_mask'], tokenized_texts['token_type_ids']]
y = df['情緒']

# Convert sentiment labels to numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train_ids, X_test_ids, X_train_attention, X_test_attention, X_train_token_type, X_test_token_type, y_train_encoded, y_test_encoded = train_test_split(
    X[0], X[1], X[2], y_encoded,
    test_size=0.25, random_state=42
)

# Convert split data to PyTorch tensors
X_train_input_ids = X_train_ids
X_test_input_ids = X_test_ids
X_train_attention_mask = X_train_attention
X_test_attention_mask = X_test_attention
X_train_token_type_ids = X_train_token_type
X_test_token_type_ids = X_test_token_type
y_train_tensor = torch.tensor(y_train_encoded)
y_test_tensor = torch.tensor(y_test_encoded)

# Print shapes of split sets to verify results
print("X_train_input_ids shape:", X_train_input_ids.shape)
print("X_test_input_ids shape:", X_test_input_ids.shape)
print("X_train_attention_mask shape:", X_train_attention_mask.shape)
print("X_test_attention_mask shape:", X_test_attention_mask.shape)
print("X_train_token_type_ids shape:", X_train_token_type_ids.shape)
print("X_test_token_type_ids shape:", X_test_token_type_ids.shape)
print("y_train_tensor shape:", y_train_tensor.shape)
print("y_test_tensor shape:", y_test_tensor.shape)

X_train_input_ids shape: torch.Size([112, 78])
X_test_input_ids shape: torch.Size([38, 78])
X_train_attention_mask shape: torch.Size([112, 78])
X_test_attention_mask shape: torch.Size([38, 78])
X_train_token_type_ids shape: torch.Size([112, 78])
X_test_token_type_ids shape: torch.Size([38, 78])
y_train_tensor shape: torch.Size([112])
y_test_tensor shape: torch.Size([38])


## 定義 bert 分類模型

在預訓練的 Bert 模型基礎上，添加一個分類層，構建一個用於情感分析的分類模型。


In [28]:
from transformers import BertForSequenceClassification

# 指定使用的預訓練模型名稱
model_name = 'bert-base-chinese'

# 定義分類類別的數量
num_labels = len(label_encoder.classes_)

# 實例化 BertForSequenceClassification 模型
bert_classifier = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

print(f"成功載入 BertForSequenceClassification 模型: {model_name}")
print(f"模型配置的分類類別數量: {bert_classifier.config.num_labels}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


成功載入 BertForSequenceClassification 模型: bert-base-chinese
模型配置的分類類別數量: 3


## 設定訓練參數

設定訓練模型的超參數，例如學習率、批次大小、訓練輪數等。


In [29]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Define hyperparameters
epochs = 3
batch_size = 8
learning_rate = 2e-5
warmup_steps = 0

# Create the optimizer
optimizer = AdamW(bert_classifier.parameters(), lr=learning_rate)

# Calculate total training steps
total_steps = len(y_train_tensor) // batch_size * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

print(f"Epochs: {epochs}")
print(f"Batch Size: {batch_size}")
print(f"Learning Rate: {learning_rate}")
print(f"Warmup Steps: {warmup_steps}")
print(f"Total Training Steps: {total_steps}")

Epochs: 3
Batch Size: 8
Learning Rate: 2e-05
Warmup Steps: 0
Total Training Steps: 42


## 訓練 bert 分類模型

使用訓練資料訓練 Bert 分類模型。


In [30]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Check for GPU and move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_classifier.to(device)
print(f"Using device: {device}")

# Create a TensorDataset from the training data
train_data = TensorDataset(X_train_input_ids, X_train_attention_mask, X_train_token_type_ids, y_train_tensor)

# Create a DataLoader for the training data
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Set the model to training mode
bert_classifier.train()

print("Starting model training...")

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Move batch to device
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Clear previous gradients
        bert_classifier.zero_grad()

        # Forward pass
        outputs = bert_classifier(b_input_ids,
                                  attention_mask=b_attention_mask,
                                  token_type_ids=b_token_type_ids,
                                  labels=b_labels)

        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(bert_classifier.parameters(), 1.0)

        # Update parameters
        optimizer.step()

        # Update the learning rate
        scheduler.step()

        # Print training progress (optional)
        if step % 10 == 0 and step != 0:
            print(f"  Batch {step}/{len(train_dataloader)} Loss: {loss.item():.4f}")

    # Calculate average loss over the epoch
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"  Average training loss: {avg_train_loss:.4f}")

print("Training complete!")

Using device: cpu
Starting model training...
Epoch 1/3
  Batch 10/14 Loss: 0.7813
  Average training loss: 0.8264
Epoch 2/3
  Batch 10/14 Loss: 0.2444
  Average training loss: 0.3834
Epoch 3/3
  Batch 10/14 Loss: 0.1300
  Average training loss: 0.2067
Training complete!


## 評估 bert 分類模型

在測試集上評估訓練好的 Bert 分類模型的效能。


In [31]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import torch

# Set the model to evaluation mode
bert_classifier.eval()

# Create a TensorDataset for the test data
test_data = TensorDataset(X_test_input_ids, X_test_attention_mask, X_test_token_type_ids, y_test_tensor)

# Create a DataLoader for the test data
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("Starting model evaluation...")

# Disable gradient calculation
with torch.no_grad():
    predictions = []
    true_labels = []

    # Iterate over the test data
    for batch in test_dataloader:
        # Move batch to device
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Forward pass
        outputs = bert_classifier(b_input_ids,
                                  attention_mask=b_attention_mask,
                                  token_type_ids=b_token_type_ids)

        # Get logits and predictions
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).flatten()

        # Store predictions and true labels
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

# Calculate accuracy
accuracy_bert = accuracy_score(true_labels, predictions)
print(f"\n使用 Bert 模型在測試集上的準確度: {accuracy_bert:.2f}")

# Generate and print classification report
report_bert = classification_report(true_labels, predictions, target_names=label_encoder.classes_)
print("\n使用 Bert 模型的分類報告:")
print(report_bert)

# Set the model back to training mode (if needed for subsequent steps)
bert_classifier.train()

Starting model evaluation...

使用 Bert 模型在測試集上的準確度: 0.92

使用 Bert 模型的分類報告:
              precision    recall  f1-score   support

          中立       0.79      1.00      0.88        11
          正面       1.00      0.80      0.89        15
          負面       1.00      1.00      1.00        12

    accuracy                           0.92        38
   macro avg       0.93      0.93      0.92        38
weighted avg       0.94      0.92      0.92        38



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 摘要
## 資料分析主要發現

Bert 模型在測試集上達到了最高的整體準確率 0.92，優於 TF-IDF + Naive Bayes (0.79) 和 TF-IDF + SVM (0.82)。

Bert 模型在分類「正面」和「負面」情緒方面表現出色，在「負面」類別上實現了完美的精確率和召回率。
對於「中立」情緒類別，Bert 模型實現了完美的召回率 (1.00)，但精確率較低 (0.85)，這表明它有時可能會將其他情緒誤判為中立。

Bert 模型的訓練顯示，平均訓練損失在各個時期（從第 1 個時期的 0.8264 到第 3 個時期的 0.2067）有所下降，表明學習成功。

## 見解與後續步驟

Bert 模型卓越的表現突顯了使用基於 Transformer 的模型來捕捉文本分類任務中複雜語義關係的優勢。
「中立」類別需要進一步分析和針對性改進，可能通過收集更多平衡的該類別數據或探索不同的模型架構或訓練策略來實現。

# 總結分析結果

分析 Bert 模型在情感分析任務上的表現，並與之前使用 TF-IDF 和傳統機器學習模型的結果進行比較。


In [33]:
# Print the comparison of accuracy scores
print("模型準確度比較:")
print(f"  Bert 模型: {accuracy_bert:.2f}")
print(f"  TF-IDF + Naive Bayes 模型: {accuracy_text2vec:.2f}")
print(f"  TF-IDF + SVM 模型: {accuracy_svm:.2f}")

print("\n分類報告比較:")
print("Bert 模型分類報告:")
print(report_bert)

print("TF-IDF + Naive Bayes 模型分類報告:")
print(report_text2vec)

print("TF-IDF + SVM 模型分類報告:")
print(report_svm)


模型準確度比較:
  Bert 模型: 0.92
  TF-IDF + Naive Bayes 模型: 0.79
  TF-IDF + SVM 模型: 0.82

分類報告比較:
Bert 模型分類報告:
              precision    recall  f1-score   support

          中立       0.79      1.00      0.88        11
          正面       1.00      0.80      0.89        15
          負面       1.00      1.00      1.00        12

    accuracy                           0.92        38
   macro avg       0.93      0.93      0.92        38
weighted avg       0.94      0.92      0.92        38

TF-IDF + Naive Bayes 模型分類報告:
              precision    recall  f1-score   support

          中立       0.58      1.00      0.73        11
          正面       1.00      0.73      0.85        15
          負面       1.00      0.67      0.80        12

    accuracy                           0.79        38
   macro avg       0.86      0.80      0.79        38
weighted avg       0.88      0.79      0.80        38

TF-IDF + SVM 模型分類報告:
              precision    recall  f1-score   support

          中立       0.65      1

Bert 模型相較於傳統方法的優勢和劣勢:
- 優勢:
  - 通常能捕捉更豐富的語義信息和上下文關係，這對於理解文本的真正含義非常有幫助。
  - 在這個任務中，Bert 模型達到了最高的整體準確率 (0.92)，顯著優於 Naive Bayes (0.79) 和 SVM (0.82)。
  - 在 '正面' 和 '負面' 情緒類別上表現出色，精確率和召回率普遍較高，尤其在 '負面' 類別上達到了完美的精確率和召回率。
- 劣勢:
  - 訓練過程需要大量的計算資源和時間。
  - 模型參數較多，訓練和推理速度相對較慢。
  - 對於 '中立' 情緒類別，雖然召回率很高 (1.00)，但精確率仍有改進空間 (0.85)，這可能表示模型容易將其他情緒誤判為中立。

本次情感分析總結與未來改進方向:
總結:
本次情感分析任務成功應用了 Bert 模型，並與傳統的 TF-IDF 結合 Naive Bayes 和 SVM 模型進行了比較。Bert 模型在整體效能上表現最佳，尤其在識別正面和負面情緒方面展現了強大的能力。
未來改進方向:
- **數據集擴充與平衡:** 雖然本次數據集情緒分佈較平均，但在更複雜或不平衡的數據集上，可以考慮增加數據量或使用過採樣/欠採樣技術。
- **模型調優:** 進一步調整 Bert 模型的超參數（例如學習率、批次大小、訓練輪數）或嘗試不同的預訓練模型。
- **Word Embedding 探索:** 雖然本次 TF-IDF 表現不錯，但 Word2Vec 等其他詞嵌入技術結合 Bert 或其他模型也值得探索。
- **模型融合:** 嘗試將不同模型的預測結果進行融合，以可能提升整體效能。
- **錯誤分析:** 深入分析模型在中立情緒類別上誤判的原因，並針對性地進行改進，例如引入更多中立情緒的樣本或進行更細緻的特徵工程。